In [1]:
import sys
sys.path.append('../')

from src.m4_generator import M4Generator
from src.m4_data_loader import M4DataLoader
from src.m4_evaluation_loss_functions import *
from src.visualization import *
from src.evaluation import *
from src.m4_model import M4Model
from glob import glob

import numpy as np
import keras
import matplotlib.pyplot as plt

import tensorflow as tf

%matplotlib inline

Using TensorFlow backend.


In [2]:
MODEL_BASE_DIR = '../models/berken'
TRAIN_PATH = "../Dataset/splitted/Hourly-train.csv"
TEST_PATH = "../Dataset/splitted/Hourly-test.csv"
TRAIN_HOLDOUT_PATH = "../Dataset/splitted/Hourly-train-holdout.csv"
TEST_HOLDOUT_PATH= "../Dataset/splitted/Hourly-test-holdout.csv"

x_augmentations = []
y_augmentations = []

models_arch = ['1-LSTM', '2-LSTM', '3-LSTM']
loss_functions = [m4_mase]

# Full Evaluation And Comparsion ( Point Prediction)
Evaluate all trained models aginst all error functions

In [3]:
data = {'Model':[], 'Training Loss Function':[], 'Epochs':[], 'LOOKBACK': [], 'Hidden Layer Size':[],
        'Features Number':[], 'T-MASE': [], 'T-MASE Naive': [], 'H-MASE': [], 'H-MASE Naive': [],
        'T-Imporvement %': [],'H-Imporvement %': [], 
       }

for model_arc in models_arch:
    s = f'{MODEL_BASE_DIR}/{model_arc}/*/'
    models = glob(s)
    for model_path in models:
        
        ev_results = {}
        
        for loss in loss_functions:
            ev_results = load_and_evaluate_model(model_path, TRAIN_PATH, TEST_PATH, 
                                                 TRAIN_HOLDOUT_PATH, TEST_HOLDOUT_PATH,
                                                 x_augmentations, y_augmentations, loss)
        data['Model'].append(model_arc)
        data['Training Loss Function'].append(ev_results['hyperparameters']['loss'])
        data['Epochs'].append(ev_results['hyperparameters']['epochs'])
        data['LOOKBACK'].append(ev_results['hyperparameters']['lookback'])
        data['Hidden Layer Size'].append(ev_results['hyperparameters']['hidden_layer_size'])
        data['Features Number'].append(ev_results['hyperparameters']['features_number'])
        data['T-MASE'].append(ev_results['test_error'])
        data['T-MASE Naive'].append(ev_results['naive_test_error'])
        data['H-MASE'].append(ev_results['holdout_error'])
        data['H-MASE Naive'].append(ev_results['naive_holdout_error'])
        
        test_naive_improv = (ev_results["naive_test_error"]- ev_results["test_error"])*100/ev_results["naive_test_error"]
        #test_snaive_improv = (ev_results["snaive_test_error"]- ev_results["test_error"])*100/ev_results["snaive_test_error"]

        holdout_naive_improv = (ev_results["naive_holdout_error"]- ev_results["holdout_error"])*100/ev_results["naive_holdout_error"]
        #holdout_snaive_improv = (ev_results["snaive_holdout_error"]- ev_results["holdout_error"])*100/ev_results["snaive_test_error"]
        
        data['T-Imporvement %'].append(round(test_naive_improv,3))
        
        #data['Test Imporvement sNaive %'].append(round(test_snaive_improv,3))
        data['H-Imporvement %'].append(round(holdout_naive_improv, 3))
        #data['holdout Imporvement sNaive %'].append(round(holdout_snaive_improv,3))

df = pd.DataFrame(data=data)

dfStyler = df.style.set_properties(**{'text-align': 'center'})
dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])


Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'


,Model,Training Loss Function,Epochs,LOOKBACK,Hidden Layer Size,Features Number,T-MASE,T-MASE Naive,H-MASE,H-MASE Naive,T-Imporvement %,H-Imporvement %
0,1-LSTM,qd_objective_lstm_c,30,652,80,1,9.276,14.552,1.539,2.642,36.256,41.749
1,1-LSTM,qd_objective_lstm_c,30,48,80,1,9.702,23.825,1.586,3.399,59.278,53.339
2,1-LSTM,qd_objective_lstm_c,100,652,80,1,9.349,14.552,1.689,2.642,35.755,36.071
3,1-LSTM,qd_objective_lstm_c,30,48,40,1,17.996,23.825,1.963,3.399,24.466,42.248
4,1-LSTM,qd_objective_lstm_c,30,48,40,1,16.458,23.825,1.588,3.399,30.921,53.28
5,1-LSTM,qd_objective_lstm_c,10,48,30,1,9.968,23.825,1.517,3.399,58.162,55.369
6,1-LSTM,qd_objective_lstm_c,30,48,40,1,17.061,23.825,1.594,3.399,28.39,53.104
7,3-LSTM,qd_objective_lstm_c,1,48,80,1,25.416,23.825,2.262,3.399,-6.678,33.451
8,3-LSTM,qd_objective_lstm_c,100,48,200,1,17.457,23.825,1.499,3.399,26.728,55.899
9,3-LSTM,qd_objective_lstm_c,10,48,120,1,17.128,23.825,1.683,3.399,28.109,50.485


In [3]:
df.to_csv('../models/berken/comparsion_point_predection.csv', sep='\t')

# Full Evaluation And Comparsion ( Intervals Prediction Test)

In [5]:
data = {'Model':[], 'Training Loss Function':[], 'Epochs':[], 'LOOKBACK': [], 'Hidden Layer Size':[],
        'Features Number':[], 'ACD': [], 'ACD Naive': [], 'MSIS': [], 'MSIS Naive': [],
        'ACD-Imporvement %': [],'MSIS-Imporvement %': [], 
       }

for model_arc in models_arch:
    s = f'{MODEL_BASE_DIR}/{model_arc}/*/'
    models = glob(s)
    for model_path in models:
        
        ev_results = {}
        
        for loss in loss_functions:
            ev_results = load_and_evaluate_model_PI(model_path, TRAIN_PATH, TEST_PATH, 
                                                 TRAIN_HOLDOUT_PATH, TEST_HOLDOUT_PATH,
                                                 x_augmentations, y_augmentations, evaluate_berken_PI)
            
        data['Model'].append(model_arc)
        data['Training Loss Function'].append(ev_results['hyperparameters']['loss'])
        data['Epochs'].append(ev_results['hyperparameters']['epochs'])
        data['LOOKBACK'].append(ev_results['hyperparameters']['lookback'])
        data['Hidden Layer Size'].append(ev_results['hyperparameters']['hidden_layer_size'])
        data['Features Number'].append(ev_results['hyperparameters']['features_number'])
        data['ACD'].append(ev_results['acd_test'])
        data['ACD Naive'].append(ev_results['acd_naive_test'])
        data['MSIS'].append(ev_results['msis_test'])
        data['MSIS Naive'].append(ev_results['msis_naive_test'])
        
        acd_naive_improv = (ev_results["acd_naive_test"]- ev_results["acd_test"])*100/ev_results["acd_naive_test"]

        msis_naive_improv = (ev_results["msis_naive_test"]- ev_results["msis_test"])*100/ev_results["msis_naive_test"]
        
        data['ACD-Imporvement %'].append(round(acd_naive_improv,3))
        data['MSIS-Imporvement %'].append(round(msis_naive_improv, 3))

df = pd.DataFrame(data=data)

dfStyler = df.style.set_properties(**{'text-align': 'center'})
dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])


Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'


,Model,Training Loss Function,Epochs,LOOKBACK,Hidden Layer Size,Features Number,ACD,ACD Naive,MSIS,MSIS Naive,ACD-Imporvement %,MSIS-Imporvement %
0,1-LSTM,qd_objective_lstm_c,30,652,80,1,0.027,0.047,120.753,58.039,42.553,-108.055
1,1-LSTM,qd_objective_lstm_c,30,48,80,1,0.021,0.047,187.266,94.136,55.319,-98.931
2,1-LSTM,qd_objective_lstm_c,100,652,80,1,0.041,0.047,80.746,58.039,12.766,-39.124
3,1-LSTM,qd_objective_lstm_c,30,48,40,1,0,0.047,199.213,94.136,100,-111.623
4,1-LSTM,qd_objective_lstm_c,30,48,40,1,0.016,0.047,322.873,94.136,65.957,-242.986
5,1-LSTM,qd_objective_lstm_c,10,48,30,1,0.01,0.047,86.134,94.136,78.723,8.5
6,1-LSTM,qd_objective_lstm_c,30,48,40,1,0.024,0.047,162.119,94.136,48.936,-72.218
7,3-LSTM,qd_objective_lstm_c,1,48,80,1,0.063,0.047,528.951,94.136,-34.043,-461.901
8,3-LSTM,qd_objective_lstm_c,100,48,200,1,0.013,0.047,376.471,94.136,72.34,-299.922
9,3-LSTM,qd_objective_lstm_c,10,48,120,1,0.002,0.047,568.321,94.136,95.745,-503.723


In [5]:
df.to_csv('../models/berken/comparsion_pi_test.csv', sep='\t')

# Full Evaluation And Comparsion ( Intervals Prediction holdout)

In [6]:
data = {'Model':[], 'Training Loss Function':[], 'Epochs':[], 'LOOKBACK': [], 'Hidden Layer Size':[],
        'Features Number':[], 'ACD': [], 'ACD Naive': [], 'MSIS': [], 'MSIS Naive': [],
        'ACD-Imporvement %': [],'MSIS-Imporvement %': [], 
       }

models_arch = ['1-LSTM', '2-LSTM', '3-LSTM']
loss_functions = [m4_mase]

for model_arc in models_arch:
    s = f'{MODEL_BASE_DIR}/{model_arc}/*/'
    models = glob(s)
    for model_path in models:
        
        ev_results = {}
        
        for loss in loss_functions:
            ev_results = load_and_evaluate_model_PI(model_path, TRAIN_PATH, TEST_PATH, 
                                                 TRAIN_HOLDOUT_PATH, TEST_HOLDOUT_PATH,
                                                 x_augmentations, y_augmentations, evaluate_berken_PI)
        data['Model'].append(model_arc)
        data['Training Loss Function'].append(ev_results['hyperparameters']['loss'])
        data['Epochs'].append(ev_results['hyperparameters']['epochs'])
        data['LOOKBACK'].append(ev_results['hyperparameters']['lookback'])
        data['Hidden Layer Size'].append(ev_results['hyperparameters']['hidden_layer_size'])
        data['Features Number'].append(ev_results['hyperparameters']['features_number'])
        data['ACD'].append(ev_results['acd_holdout'])
        data['ACD Naive'].append(ev_results['acd_naive_holdout'])
        data['MSIS'].append(ev_results['msis_holdout'])
        data['MSIS Naive'].append(ev_results['msis_naive_holdout'])
        
        acd_naive_improv = (ev_results["acd_naive_holdout"]- ev_results["acd_holdout"])*100/ev_results["acd_naive_holdout"]

        msis_naive_improv = (ev_results["msis_naive_holdout"]- ev_results["msis_holdout"])*100/ev_results["msis_naive_holdout"]
        
        data['ACD-Imporvement %'].append(round(acd_naive_improv,3))
        data['MSIS-Imporvement %'].append(round(msis_naive_improv, 3))

df = pd.DataFrame(data=data)

dfStyler = df.style.set_properties(**{'text-align': 'center'})
dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])


Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'
Loaded model from disk
Missing key'clipvalue'


,Model,Training Loss Function,Epochs,LOOKBACK,Hidden Layer Size,Features Number,ACD,ACD Naive,MSIS,MSIS Naive,ACD-Imporvement %,MSIS-Imporvement %
0,1-LSTM,qd_objective_lstm_c,30,652,80,1,0.023,0.042,20.263,10.082,45.238,-100.982
1,1-LSTM,qd_objective_lstm_c,30,48,80,1,0.012,0.042,22.145,12.779,71.429,-73.292
2,1-LSTM,qd_objective_lstm_c,100,652,80,1,0.028,0.042,14.136,10.082,33.333,-40.21
3,1-LSTM,qd_objective_lstm_c,30,48,40,1,0.022,0.042,26.634,12.779,47.619,-108.42
4,1-LSTM,qd_objective_lstm_c,30,48,40,1,0.021,0.042,39.493,12.779,50,-209.046
5,1-LSTM,qd_objective_lstm_c,10,48,30,1,0.013,0.042,13.894,12.779,69.048,-8.725
6,1-LSTM,qd_objective_lstm_c,30,48,40,1,0.041,0.042,24.33,12.779,2.381,-90.39
7,3-LSTM,qd_objective_lstm_c,1,48,80,1,0.063,0.042,68.582,12.779,-50,-436.677
8,3-LSTM,qd_objective_lstm_c,100,48,200,1,0.015,0.042,45.703,12.779,64.286,-257.641
9,3-LSTM,qd_objective_lstm_c,10,48,120,1,0.006,0.042,72.513,12.779,85.714,-467.439


In [10]:
df.to_csv('../models/berken/comparsion_pi_holdout.csv', sep='\t')